In [ ]:
import os, json, csv
from datetime import datetime
import pandas as pd

In [ ]:
outputDirectory = '../20190722-162512-slack_export'

class args:
    channel_prefix = '3_'
    deadline = '2019-07-22'

In [ ]:
## ---------------------------- Get users url data by reactions ---------------------------- ##
# Get user data
users = {}
with open(os.path.join(outputDirectory, 'users.json')) as json_file:
    user_json = json.load(json_file)
    for user in user_json:
        if not (user['is_bot'] or (user['name'] == 'slackbot')):
            if 'real_name' in user.keys():
                users[user['id']] = user['real_name']
            else:
                users[user['id']] = user['profile']['real_name']

# Get Filtered URLs from saved data
# need dataz only from [3_*] channels
all_channels = os.listdir(outputDirectory)
filtered_channels = sorted([i for i in all_channels if i.startswith(args.channel_prefix)])

# collect all data from filtered channels
all_messages = []
for channel in filtered_channels:
    channel_path = os.path.join(outputDirectory, channel)
    for date in sorted(os.listdir(channel_path)):
        with open(os.path.join(channel_path, date)) as json_file:
            json_data = json.load(json_file)
            all_messages.extend(json_data)

def self_reaction_check(check_reaction, message):
    '''
    메세지에 self로 check_reaction을 입력했는지 확인
    '''
    reactions = message['reactions']
    userId = message['user']
    for reaction in reactions:
        if reaction['name'] == check_reaction:
            if userId in reaction['users']:
                return True
            else:
                return False
        else:
            return False

# filter data with reactions
submit_reaction = 'submit'
pass_reaction = 'pass'
submit_data = pd.DataFrame({'userId': userid, 'url': -1, 'time': None} \
                           for userid in users)
for message in all_messages:
    # 1) submit
    if ('attachments' in message.keys()) and ('reactions' in message.keys()):
        if self_reaction_check(submit_reaction, message):
            userId = message['user']
            link = message['attachments'][0]['title_link']
            time = str(datetime.fromtimestamp(float(message['ts'])))[:-7]

            submit_data.loc[submit_data['userId'] == userId, 'url'] = link
            submit_data.loc[submit_data['userId'] == userId, 'time'] = time

    # 2) pass
    if ('reactions' in message.keys()) and (self_reaction_check(pass_reaction, message)):
        userId = message['user']
        time = str(datetime.fromtimestamp(float(message['ts'])))[:-7]
        submit_data.loc[submit_data['userId'] == userId, 'url'] = 'pass'
        submit_data.loc[submit_data['userId'] == userId, 'time'] = time


### send user data to BigQuery

In [ ]:
users_df = pd.read_csv('../user_data.csv')

In [ ]:
name_to_id = {}
for userid in users.keys():
    name_to_id[users[userid]] = userid

name_to_id

In [ ]:
users_df['id'] = users_df['name'].apply(lambda i: name_to_id[i])
users_df

In [ ]:
users_df.to_gbq(table_id, project_id=project_id, if_exists='replace')

### send slack log data to BigQuery

In [ ]:
submit_data

### deadline_check

In [ ]:
from datetime import datetime, timedelta

In [ ]:
deadline_str = args.deadline
deadline_str

In [ ]:
deadline_time = datetime.strptime(deadline_str, '%Y-%m-%d')
deadline_time

In [ ]:
submit_deadline = deadline_time + timedelta(hours=2)
submit_deadline

In [ ]:
submit_time_str = '2019-07-21 23:21:24'
submit_time = datetime.strptime(submit_time_str, '%Y-%m-%d %H:%M:%S')
submit_time

In [ ]:
submit_time < deadline_time

### status table

In [ ]:
query = '''
select url, name
from `geultto.slack_log.3rd_staging` as l left outer join `geultto.user_db.team_member` as r
on l.userId = r.id
'''

In [ ]:
project_id = 'geultto'
slack_log = pd.read_gbq(query, project_id=project_id, dialect='standard')
slack_log